In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision.datasets import MNIST
from torchvision.transforms import Compose,ToTensor,Normalize
from torch.utils.data import DataLoader
torch.__version__

'1.13.0+cu117'

In [2]:
BATCH_SIZE=512
#大概需要2G的显存
EPOCHS=2 
# 总共训练批次
gpu_abailable=torch.cuda.is_available()
print(gpu_abailable)
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu") 
# 判断是否使用GPU

True


In [3]:
def get_dataloader(train,batch_size=BATCH_SIZE):
    transform_fn = Compose([
        ToTensor(),
        Normalize(mean = (0.1307,),std = (0.3081,))
        ])#mean和std的形状与通道数相同
 
    dataset = MNIST(download=True,root = './data',train = train,transform = transform_fn)
    data_loader = DataLoader(dataset,batch_size = batch_size,shuffle = True)
    return data_loader

In [4]:
train_loader=get_dataloader(True)
train_loader

In [5]:
test_loader=get_dataloader(False)
train_loader

In [6]:
trainloader = torch.utils.data.DataLoader(
        datasets.MNIST('data', train=True, download=True, 
                       transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=BATCH_SIZE, shuffle=True)

In [7]:
testloader = torch.utils.data.DataLoader(
        datasets.MNIST('data', train=False, transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=BATCH_SIZE, shuffle=True)

In [8]:
# Create ANN Model
class ANNModel(nn.Module):
     
    def __init__(self):
        super(ANNModel, self).__init__()
         
        # Linear function 1: 784 --> 150
        self.fc1 = nn.Linear(1*28*28, 28)
         
        # Linear function 2: 150 --> 10
        self.fc2 = nn.Linear(28, 10)
        
    
    def forward(self, input):
        #print('input size:',input.shape)
        x = input.view([-1,1*28*28])
        #print('input size:',x.shape)
        # Linear function 1
        x = self.fc1(x)
        #print('input size:',x.shape)
        # Non-linearity 1
        x = F.relu(x)
        #print('input size:',x.shape)
         
        # Linear function 2
        out = self.fc2(x)
        #print('input size:',out.shape)
        
        out = F.log_softmax(out,dim=1)
   
        return out

In [9]:
# 连接到GPU
model = ANNModel().to(DEVICE)
# 创建优化器并传入参数
#optimizer = optim.Adam(model.parameters())
learning_rate = 0.02
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)



In [10]:
def train(model, device, train_loader, optimizer, epoch):
    # 将模型转为train模式
    model.train()
    for batch_idx, (data,target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        # 重置优化器梯度
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output,target)
        loss.backward()
        optimizer.step()
        if((batch_idx)%30)==0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

In [11]:
def test(model,device,test_loader):
    # 将模型转为test模式
    model.eval()
    test_loss=0
    correct=0
    with torch.no_grad():
        for data,target in test_loader:
            data,target = data.to(device),target.to(device)
            output = model(data)
            test_loss+= F.nll_loss(output,target,reduction='sum').item()
            pred = output.max(1,keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()
    
    test_loss/=len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
            

In [12]:
for epoch in range(1,EPOCHS+1):
    train(model,DEVICE,train_loader,optimizer,epoch)
    test(model,DEVICE,test_loader)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.297929
Train Epoch: 1 [15360/60000 (25%)]	Loss: 1.531602
Train Epoch: 1 [30720/60000 (51%)]	Loss: 1.020163
Train Epoch: 1 [46080/60000 (76%)]	Loss: 0.750876

Test set: Average loss: 0.5919, Accuracy: 8612/10000 (86%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.587130
Train Epoch: 2 [15360/60000 (25%)]	Loss: 0.545383
Train Epoch: 2 [30720/60000 (51%)]	Loss: 0.432324
Train Epoch: 2 [46080/60000 (76%)]	Loss: 0.385230

Test set: Average loss: 0.4139, Accuracy: 8911/10000 (89%)

